# Data Visualization with py-gdelt

Visualizing GDELT data with charts and maps.

## Contents
1. Timeline Charts (matplotlib/plotly)
2. Geographic Maps (folium)
3. Tone/Sentiment Analysis Plots
4. TV Station Comparison Charts

In [ ]:
# Install visualization dependencies if needed
# !pip install pandas matplotlib folium plotly

# Setup
import nest_asyncio


nest_asyncio.apply()


from py_gdelt import GDELTClient


# Import visualization libraries with fallback
try:
    import folium
    import matplotlib.pyplot as plt
    import pandas as pd

    HAS_VIZ = True
except ImportError as e:
    print(f"Missing visualization libraries: {e}")
    print("Install with: pip install pandas matplotlib folium")
    HAS_VIZ = False

## 1. Timeline Charts

Visualize trends over time using DOC API timelines:

In [ ]:
if HAS_VIZ:
    async with GDELTClient() as client:
        try:
            # Get timeline data
            timeline = await client.doc.timeline(
                query="climate change",
                timespan="7d",
            )

            if timeline.points:
                # Convert to DataFrame
                df = pd.DataFrame(
                    [{"date": point.date, "count": point.count} for point in timeline.points],
                )

                # Plot
                plt.figure(figsize=(12, 6))
                plt.plot(df["date"], df["count"], marker="o")
                plt.title("Climate Change Mentions Over Time")
                plt.xlabel("Date")
                plt.ylabel("Article Count")
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()

                print(f"Total articles: {df['count'].sum()}")
            else:
                print("No timeline data available")

        except Exception as e:
            print(f"Error: {e}")
else:
    print("Visualization libraries not available")

## 2. Geographic Maps

Visualize geographic data using folium:

In [ ]:
if HAS_VIZ:
    async with GDELTClient() as client:
        try:
            # Get geographic points
            result = await client.geo.search(
                "earthquake",
                timespan="7d",
                max_points=50,
            )

            if result.points:
                # Create map centered on mean position
                mean_lat = sum(p.lat for p in result.points) / len(result.points)
                mean_lon = sum(p.lon for p in result.points) / len(result.points)

                m = folium.Map(location=[mean_lat, mean_lon], zoom_start=2)

                # Add markers
                for point in result.points[:30]:  # Limit to 30 for performance
                    folium.CircleMarker(
                        location=[point.lat, point.lon],
                        radius=min(point.count / 10, 20),  # Scale by count
                        popup=f"{point.name}: {point.count} articles",
                        color="red",
                        fill=True,
                        fillColor="red",
                        fillOpacity=0.6,
                    ).add_to(m)

                display(m)
            else:
                print("No geographic data available")

        except Exception as e:
            print(f"Error: {e}")
else:
    print("Visualization libraries not available")

## 3. Tone/Sentiment Analysis

Visualize sentiment from Context API:

In [ ]:
if HAS_VIZ:
    async with GDELTClient() as client:
        try:
            # Get context analysis
            result = await client.context.analyze(
                "economy",
                timespan="7d",
            )

            if result.tone:
                # Create bar chart of tone distribution
                tone_data = {
                    "Positive": result.tone.positive_count,
                    "Negative": result.tone.negative_count,
                    "Neutral": result.tone.neutral_count,
                }

                plt.figure(figsize=(10, 6))
                plt.bar(tone_data.keys(), tone_data.values(), color=["green", "red", "gray"])
                plt.title(f"Tone Distribution for 'economy' (Avg: {result.tone.average_tone:.2f})")
                plt.ylabel("Article Count")
                plt.tight_layout()
                plt.show()

                print(f"Total articles analyzed: {result.article_count}")
            else:
                print("No tone data available")

        except Exception as e:
            print(f"Error: {e}")
else:
    print("Visualization libraries not available")

## 4. TV Station Comparison

Compare coverage across TV stations:

In [ ]:
if HAS_VIZ:
    async with GDELTClient() as client:
        try:
            # Get station comparison
            chart = await client.tv.station_chart(
                "healthcare",
                timespan="7d",
            )

            if chart.stations:
                # Get top 10 stations
                top_stations = chart.stations[:10]

                stations = [s.station for s in top_stations]
                counts = [s.count for s in top_stations]

                # Create horizontal bar chart
                plt.figure(figsize=(12, 6))
                plt.barh(stations, counts)
                plt.xlabel("Mention Count")
                plt.title("TV Station Coverage: 'healthcare'")
                plt.tight_layout()
                plt.show()

                print(f"Total mentions: {sum(counts)}")
            else:
                print("No station data available")

        except Exception as e:
            print(f"Error: {e}")
else:
    print("Visualization libraries not available")

## Summary

You've learned:
- ✅ Creating timeline charts from DOC API
- ✅ Mapping geographic data with folium
- ✅ Visualizing sentiment analysis
- ✅ Comparing TV station coverage

**Visualization Tips:**
- Use `pandas` for data manipulation
- `matplotlib` for static charts
- `plotly` for interactive charts
- `folium` for geographic maps
- Always handle missing data gracefully
- Limit map markers for performance (30-50 max)